In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import compute_barycenter

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time

/home/octav/gitrepos/tum-thesis


In [2]:
sys.argv = ["", "-cuda", "-pred_hidden", "150"]
args = get_args()
args.n_epochs = 40 
args.output_dir = "mol_opt/output"
args.init_model = "transformer1"
args.init_decoder_model = "{}_decode".format(args.init_model)
args

Namespace(N_transformer=6, agg_func='sum', batch_norm=False, cuda=True, device='cuda:0', dim_tangent_space=40, dropout_ffn=0.0, dropout_gcn=0.0, dropout_transformer=0.1, ffn_activation='LeakyReLU', init_decoder_model='transformer1_decode', init_model='transformer1', linear_out=False, n_epochs=40, n_ffn_hidden=100, n_ffn_transformer=100, n_heads_transformer=10, n_hidden=50, n_labels=1, n_layers=5, ot_solver='emd', output_dir='mol_opt/output', pc_hidden=50, pred_hidden=150, sinkhorn_entropy=0.1, sinkhorn_max_it=10000)

In [3]:
train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train", 96, False)
val_data_loader = get_loader("iclr19-graph2graph/data/qed", "val", 96, False)

In [ ]:
molopt = main(args, train_data_loader = train_data_loader, val_data_loader = val_data_loader)

No model transformer1 found in mol_opt/output! Starting from scratch.
No model transformer1_decode found in mol_opt/output! Starting from scratch.
Epoch: 1


/home/octav/gitrepos/tum-thesis/otgnn/models/gromov_modules.py:394: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  nce_reg = torch.nn.LogSoftmax()(torch.stack(all_nce_dists))[0]
